In [3]:
!pip install joblib

Defaulting to user installation because normal site-packages is not writeable


In [4]:
from joblib import Parallel, delayed

In [6]:
import numpy as np
import xarray as xr

In [7]:
data_path = '/glade/u/home/cassiacai/marine_heatwaves/notebooks/SSTA_and_events_0_3.nc'

In [8]:
# read as dask array when chunks are specified
events = xr.open_dataset(data_path)

In [17]:
# simple function returning one scalar
def calc_cumulativeintensity(events, mhw_id):
    cumulative_intensity = np.nansum(events.where(events.labels==mhw_id, drop=True).SSTA)
    return cumulative_intensity

**Run without parallelization:**

In [18]:
%%time
measures = [calc_cumulativeintensity(events, mhw_id) for mhw_id in list(range(1,1203))]

CPU times: user 5min 57s, sys: 4min 10s, total: 10min 7s
Wall time: 10min 7s


**Run with parallelization:**

In [19]:
%%time
measures = Parallel(n_jobs=8, require="sharedmem")(delayed(calc_cumulativeintensity)(events, mhw_id) for mhw_id in list(range(1,1203)))

CPU times: user 6min 41s, sys: 4min 18s, total: 10min 59s
Wall time: 1min 27s
